In [6]:
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import IntText, Text
import ipywidgets as widgets
from numpy.random import randint
import numpy as np
map = lambda xs, f: [f(x) for x in xs]
join = lambda xs, sep: sep.join(xs)

def mod(a, b):
    return a - b * (a // b)

### Parametry
* `p` liczba pierwsza
* `t` wymagana liczba udziałów
* `n` całkowita liczba udziałów

In [3]:
# Liczba pierwsza
p = 2**13 - 1
p 

8191

In [4]:
def poly_eval(A, x):
    X = np.array([x**i for i in range(len(A))])
    y = np.sum(A*X)
    return y

@interact_manual(secret=IntText(), n=(1, 10, 1), t=(1, 10, 1))
def shamir_encrypt(secret, n=4, t=3):
    assert p > secret, f'p > secret not satisfied ({p} <= {secret})'
    assert p > n, f'p > n not satisfied ({p} <= {n})'
    A = [secret] + list(randint(0, p, size=t-1))
    X = np.arange(n) + 1
    Y = [poly_eval(A, x) for x in X]
    
    # Print results
    for i, x in enumerate(A): print(f'a_{i} = {x}')
    for x, y in zip(X, Y): print(f's_{x} = f({x}) mod p = {y: 8d} mod {p} = {y % p}')
    print('--- ENCRYPTED ---')
    print(join([f'{x},{y % p}' for x, y in zip(X, Y)], ';'))

interactive(children=(IntText(value=0, description='secret'), IntSlider(value=4, description='n', max=10, min=…

In [5]:
from fractions import Fraction

def lagr_interp(X):
    num, den = np.ones_like(X), np.ones_like(X)
    for i, xi in enumerate(X):
        for j, xj in enumerate(X):
            if i == j: continue
            num[i] *= xj
            den[i] *= xj - xi
    return [Fraction(a, b) for a, b in zip(num, den)]

@interact_manual(keys=Text())
def shamir_decrypt(keys):
    X, Y = np.array([map(x.split(','), int) for x in keys.split(';')]).T
    L = lagr_interp(X)
    S = np.sum(Y * L) % p
    
    # pretty print results
    for i in range(len(X)): print(f'y_{i}*l_{i} = {Y[i]: 6d} * {L[i]} = {Y[i] * L[i]}')
    print('--- DECRYPTED ---')
    print(S)

interactive(children=(Text(value='', description='keys'), Button(description='Run Interact', style=ButtonStyle…